# SFRD from IR based data

Until now, we have been using UV observations to determine the luminosity functions and SFRDs from it. Now, let's turn our attention to the IR data. We currently have three references for such observations. We, for now, focus on these data. We try to propagate the errors correctly. Below, we show this computation. We also save the computed SFRDs from IR data for further uses.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as con
import os
import utils as utl
import irlf as irlf

In [2]:
# Defining kappa and luminosity range

kap_ir = 4.5*10**(-44)
lums_ir1 = np.logspace(6, 15, 10000)*con.L_sun.value*1e7


# Making a list of IR data
p1 = os.getcwd() + '/data/New_IR/'
p2 = os.getcwd() + '/Results/'

list1 = os.listdir(p1)

In [7]:
plt.figure(figsize=(16,9))

f33 = open(p2 + 'sfrd_ir_new.dat','w')
f33.write('#Name_of_the_paper\tZ_down\tZ_up\tSFRD\tSFRD_err\n')

for i in range(len(list1)):
    z1_ir, z2_ir, alp_ir, alperr_ir, lst_ir, lsterr_ir, sig_ir, sigerr_ir, phi_ir, phierr_ir = np.loadtxt(p1 + list1[i], usecols=(0,1,2,3,4,5,6,7,8,9), unpack=True)
    ppr_n1 = np.loadtxt(p1 + list1[i], usecols=10, dtype=str, unpack=True)
    if type(alp_ir) == np.float64:
        z1_ir, z2_ir, alp_ir, alperr_ir, lst_ir, lsterr_ir, sig_ir, sigerr_ir, phi_ir, phierr_ir, ppr_n1 = np.array([z1_ir]), np.array([z2_ir]), np.array([alp_ir]), np.array([alperr_ir]), np.array([lst_ir]), np.array([lsterr_ir]), np.array([sig_ir]), np.array([sigerr_ir]), np.array([phi_ir]), np.array([phierr_ir]), np.array([ppr_n1])
    sfrd_ir12, sfrd_ir12_err = np.array([]), np.array([])
    for j in range(len(z1_ir)):
        sfrd_ir, sfrd_err_ir = irlf.sfrd_w_err(lum=lums_ir1, lst9=lst_ir[i], lst9err=lsterr_ir[i], phi9=phi_ir[i], phi9err=phierr_ir[i], sig9=sig_ir[i], sig9err=sigerr_ir[i], alp9=alp_ir[i], alp9err=alperr_ir[i], kappa=kap_ir, limit=0.03)
        sfrd_ir12 = np.hstack((sfrd_ir12, sfrd_ir))
        sfrd_ir12_err = np.hstack((sfrd_ir12_err, sfrd_err_ir))
        f33.write(ppr_n1[0] + '\t' + str(z1_ir[j]) + '\t' + str(z2_ir[j]) + '\t' + str(sfrd_ir) + '\t' + str(sfrd_err_ir) + '\n')
    zcen1 = (z1_ir + z2_ir)/2
    log_sfrd_ir12, log_sfrd_ir12_err = utl.log_err(sfrd_ir12, sfrd_ir12_err)
    plt.errorbar(zcen1, log_sfrd_ir12, log_sfrd_ir12_err, label=ppr_n1[i])

f33.close()
plt.xlabel('Redshift')
plt.ylabel('SFRD (in M_sun year^-1 Mpc^-3)')
plt.grid()
plt.legend(loc='best')

  4%|▍         | 420/10000 [00:04<01:34, 101.83it/s]


KeyboardInterrupt: 